In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [16]:
tf.__version__

'2.4.1'

In [8]:
# !pip install tensorflow_datasets
tf.version

<module 'tensorflow._api.v2.version' from 'C:\\Users\\Abir\\AppData\\Roaming\\Python\\Python37\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

#### DOWNLOADING AND PREPROCESSING THE DATA

In [60]:
BUFFER_SIZE = 70000
BATCH_SIZE  = 128  ## A considerable amount of batch size helps to speed up the training process. 
                   ## On the other hands lower batch size gives better test accuray. It is desirable to have batch size of power of two.
                   ## For example, 32, 64, 128 etc.
NUM_EPOCHS  = 20

In [52]:
mnist_datasets, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

In [53]:
mnist_train , mnist_test = mnist_datasets['train'] , mnist_datasets['test']

In [46]:
## normalize the data between (0 and 1)

def normalize(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image,label

In [47]:
train_and_validation_data = mnist_train.map(normalize)
test_data = mnist_test.map(normalize)

In [48]:
## Split in validation set and test set
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

In [49]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

In [55]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [59]:
## Train and test Splitting

train_data      = train_and_validation_data.skip(num_validation_samples) ## Do not take first n(=num_validation_samples) samples
validation_data = train_and_validation_data.take(num_validation_samples) ## Take first n(=num_validation_samples) samples. 
                                                                         ## Just opposite of skip.

In [63]:
train_data = train_data.batch(BATCH_SIZE)

In [65]:
## Though batching is not required for validation and test data as we are not going to do backward propagation on them. 
# But, it is required to have the same dimension so need to create a dummy batch.

validation_data = validation_data.batch(num_validation_samples)
test_data       = test_data.batch(num_test_samples)

## Creating the Model and Training IT

In [67]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,5,activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(50,3,activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
]
)

In [68]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 50)        1300      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 50)        22550     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12510     
Total params: 36,360
Trainable params: 36,360
Non-trainable params: 0
____________________________________________________